In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

get_ipython().run_line_magic("matplotlib", "inline")


In [2]:
nltk.download("vader_lexicon")
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\14694\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Load .env enviroment variables
load_dotenv()


# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [4]:
def stock_info_grab(ticker):
    """
    Takes ticker symbol and returns DataFrame with Date, Close, and Pct Change columns.
    """
    # Set timeframe to '1D'
    timeframe = "1D"

    # Set current date and the date from one month ago using the ISO format
    current_date = pd.Timestamp("2020-11-09", tz="America/New_York").isoformat()
    past_date = pd.Timestamp("2016-08-27", tz="America/New_York").isoformat()

    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=past_date,
        end=current_date,
        after=None,
        until=None,
    ).df
    df = df.droplevel(axis=1, level=0)
    df.index = df.index.date
    df['pct change'] = df['close'].pct_change()
    df['pct change'].dropna
    df = df.reset_index()
    df = df.drop(columns=['open', 'high', 'low', 'volume'])
    df = df.rename(columns={'index':'Date'})
    df = df.set_index('Date')
    return df

In [5]:
aapl_stock_info = stock_info_grab("AAPL")
amzn_stock_info = stock_info_grab("AMZN")
tsla_stock_info = stock_info_grab("TSLA")
spy_stock_info = stock_info_grab("SPY")
aapl_stock_info

,close,pct change
Date,,
2016-08-29,106.820,NaN
2016-08-30,105.990,-0.007770
2016-08-31,106.110,0.001132
2016-09-01,106.730,0.005843
2016-09-02,107.730,0.009369
...,...,...
2020-11-03,110.375,0.014756
2020-11-04,114.940,0.041359
2020-11-05,118.990,0.035236


In [15]:
aapl_file = Path('Resources/AAPL_HEADLINES.csv')
amzn_file = Path('Resources/AMZN_HEADLINES.csv')
spy_file = Path('Resources/SPY_HEADLINES.csv')
tsla_file = Path('Resources/TSLA_HEADLINES.csv')

aapl_headlines_df = pd.read_csv(aapl_file)
amzn_headlines_df = pd.read_csv(amzn_file)
spy_headlines_df = pd.read_csv(spy_file)
tsla_headlines_df = pd.read_csv(tsla_file)

#aapl_headlines['Date'] = pd.to_datetime(aapl_headlines['Date']).dt.strftime('%Y-%m-%d')
#aapl_headlines = aapl_headlines.set_index('Date')
amzn_headlines_df

,Headline,Date
0,Here are Wall Street’s 20 favorite value stocks,"Nov. 10, 2020 at 12:53 p.m. ET"
1,"Amazon ‘illegally distorted competition,’ the ...","Nov. 14, 2020 at 3:33 a.m. ET"
2,Mary Barra on General Motors’ All-Electric Future,"Nov. 13, 2020 at 8:39 p.m. ET"
3,Target Is Booming During the Pandemic. Why the...,"Nov. 13, 2020 at 8:38 p.m. ET"
4,These 5 Small Stocks Could Benefit From an Eco...,"Nov. 13, 2020 at 8:00 p.m. ET"
...,...,...
19631,Elsa to Barbie: Let it Go,"Nov. 25, 2014 at 12:36 p.m. ET"
19632,Tony Robbins doesn’t quite master the game of ...,"Nov. 25, 2014 at 11:23 a.m. ET"
19633,How banks are looking more like tech companies,"Nov. 25, 2014 at 9:24 a.m. ET"
19634,"Is the S&P’s future healthy after all, and a ‘...","Nov. 25, 2014 at 9:04 a.m. ET"


In [16]:
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result


In [17]:
def create_sentiment_df(df):
    """
    Takes headlines DataFrame & creates DataFrame with Sentiment columns.
    Splits Date & Time, creates Time column and moves Date to Index.
    """
    title_sent = {
        "compound": [],
        "positive": [],
        "neutral": [],
        "negative": [],
        "sentiment": [],
    }

    for index, row in df.iterrows():
        try:
            # Sentiment scoring with VADER
            title_sentiment = analyzer.polarity_scores(row["Headline"])
            title_sent["compound"].append(title_sentiment["compound"])
            title_sent["positive"].append(title_sentiment["pos"])
            title_sent["neutral"].append(title_sentiment["neu"])
            title_sent["negative"].append(title_sentiment["neg"])
            title_sent["sentiment"].append(get_sentiment(title_sentiment["compound"]))
        except AttributeError:
            pass

    title_sent_df = pd.DataFrame(title_sent)
    #title_sent_df.head()

    headline_sentiment_df = df.join(title_sent_df)
    headline_sentiment_df.dropna()
    headline_sentiment_df['Date'] = headline_sentiment_df['Date'].str.replace('at','-')
    headline_sentiment_df['Date'] = headline_sentiment_df['Date'].str.split('-').str[0]
    headline_sentiment_df = headline_sentiment_df.reindex(columns=['Date', 'Headline', 'compound', 'positive', 'neutral', 'negative', 'sentiment'])
    headline_sentiment_df['Date'] = pd.to_datetime(headline_sentiment_df['Date'])
    headline_sentiment_df.set_index('Date')
    return headline_sentiment_df

In [31]:
#issue with amzn_headlines --- need to fix
aapl_headlines = create_sentiment_df(aapl_headlines_df)
#amzn_headlines = create_sentiment_df(amzn_headlines_df)
tsla_headlines = create_sentiment_df(tsla_headlines_df)
spy_headlines = create_sentiment_df(spy_headlines_df)
aapl_headlines

,Date,Headline,compound,positive,neutral,negative,sentiment
0,2020-11-09,"Apple Inc. stock falls Monday, underperforms m...",0.0000,0.000,1.000,0.000,0
1,2020-11-09,Big Tech Stocks Are Lagging Today. Why They’ll...,-0.0772,0.121,0.738,0.141,-1
2,2020-11-09,"As Apple releases its new line of Macs, the bi...",0.4767,0.193,0.807,0.000,1
3,2020-11-06,"In the Midst of Election Uncertainty, Younger ...",-0.3400,0.000,0.806,0.194,-1
4,2020-11-07,Berkshire Buybacks Hit Record $9 Billion in Th...,-0.1531,0.000,0.882,0.118,-1
...,...,...,...,...,...,...,...
9868,2016-08-29,Respect for America has climbed during the Oba...,0.4767,0.279,0.721,0.000,1
9869,2016-08-29,"Fitbit upgrades now track yoga, weightlifting ...",0.0000,0.000,1.000,0.000,0
9870,2016-08-28,5 things Tim Cook has done better at Apple tha...,0.4404,0.209,0.791,0.000,1
9871,2016-08-27,Want to invest in self-driving cars? Check out...,0.0772,0.126,0.874,0.000,1


In [30]:
aapl_headlines

In [32]:
# find average sentiment score by date
aapl_scores = aapl_headlines.groupby('Date').mean().sort_values(by='Date')
#amzn_scores = amzn_headlines.groupby(['Date']).mean().sort_values(by='Date')
tsla_scores = tsla_headlines.groupby(['Date']).mean().sort_values(by='Date')
spy_scores = spy_headlines.groupby(['Date']).mean().sort_values(by='Date')

In [29]:
# TO DO: drop compund col on all scores
aapl_scores = aapl_scores.drop(columns='compound', inplace=True)
#amzn_scores = amzn_scores.drop(columns='compound', inplace=True)
tsla_scores = tsla_scores.drop(columns='compound', inplace=True)
spy_scores = spy_scores.drop(columns='compound', inplace=True)

In [68]:
# sent scores distribution across each df poss use histogram, calc meanstd, or percentiles 
aapl_complete = pd.concat([aapl_scores,aapl_stock_info], join='outer', axis=1).dropna()
#amzn_complete = pd.concat([amzn_scores,amzn_stock_info], join='outer', axis=1).dropna()
tsla_complete = pd.concat([tsla_scores,tsla_stock_info], join='outer', axis=1).dropna()
spy_complete = pd.concat([spy_scores,spy_stock_info], join='outer', axis=1).dropna()
aapl_complete

,positive,neutral,negative,sentiment,close,pct change
Date,,,,,,
2016-08-30,0.061591,0.883455,0.054955,-0.090909,105.990,-0.007770
2016-08-31,0.070400,0.818600,0.111000,-0.200000,106.110,0.001132
2016-09-01,0.069625,0.897625,0.032750,0.125000,106.730,0.005843
2016-09-02,0.063143,0.845429,0.091429,-0.285714,107.730,0.009369
2016-09-06,0.131750,0.804500,0.063750,0.250000,107.700,-0.000278
...,...,...,...,...,...,...
2020-11-03,0.119000,0.842000,0.038833,0.500000,110.375,0.014756
2020-11-04,0.078900,0.800900,0.120300,-0.300000,114.940,0.041359
2020-11-05,0.202333,0.747333,0.050333,0.333333,118.990,0.035236


In [69]:
# TO DO: shift aapl_complete['pct change'] one day on all dfs
# TO DO: dropna() on all df['predicted pct change'] cols 
aapl_complete['predicted pct change'] = aapl_complete['pct change'].shift(periods=-1)
#amzn_complete['predicted pct change'] = amzn_complete['pct change'].shift(periods=-1)
tsla_complete['predicted pct change'] = tsla_complete['pct change'].shift(periods=-1)
spy_complete['predicted pct change'] = spy_complete['pct change'].shift(periods=-1)


In [70]:
aapl_complete = aapl_complete.dropna()

In [71]:
# DO NOT USE: Not complete
def get_sentiment(df):
    """
    Calculates the sentiment based on the compound score.
    """
    result = []  # Neutral by default
    for sent in df['sentiment']:
        if sent >= 0.10: #Sell
            result.append('sell')
        elif -0.10 < sent < 0.10: #Hold
            result.append('hold')
        elif sent <= -0.10: #Buy
            result.append('buy')
            
    df['buy/hold/sell'] = pd.Series(result).values
    
    #column_values = 
    #df.insert(loc=1, column='no', value=column_values)

    return df

aapl_complete_sentiment = get_sentiment(aapl_complete)

<ipython-input-71-2491d023ff00>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['buy/hold/sell'] = pd.Series(result).values


,positive,neutral,negative,sentiment,close,pct change,predicted pct change,buy/hold/sell
Date,,,,,,,,
2016-08-30,0.061591,0.883455,0.054955,-0.090909,105.990,-0.007770,0.001132,hold
2016-08-31,0.070400,0.818600,0.111000,-0.200000,106.110,0.001132,0.005843,buy
2016-09-01,0.069625,0.897625,0.032750,0.125000,106.730,0.005843,0.009369,sell
2016-09-02,0.063143,0.845429,0.091429,-0.285714,107.730,0.009369,-0.000278,buy
2016-09-06,0.131750,0.804500,0.063750,0.250000,107.700,-0.000278,0.006221,sell
...,...,...,...,...,...,...,...,...
2020-11-02,0.038600,0.896400,0.065000,-0.200000,108.770,-0.001194,0.014756,buy
2020-11-03,0.119000,0.842000,0.038833,0.500000,110.375,0.014756,0.041359,sell
2020-11-04,0.078900,0.800900,0.120300,-0.300000,114.940,0.041359,0.035236,buy


In [ ]:
aapl_complete_sentiment = get_sentiment(aapl_complete)

In [ ]:
#not relevant, using as template for final df
df = pd.DataFrame(columns=['Date',
                           'Time',
                           'Headline',
                           'Vader compound',
                           'Vader positive',
                           'Vader neutral',
                           'Vader negative',
                           'Vader sentiment',
                           'Lex compound',
                           'Lex positive',
                           'Lex neutral',
                           'Lex negative',
                           'Lex sentiment',
                           'Sentiment Difference',
                           'stock close',
                           'stock pct change',
                           'buy/sell/hold'])